In [21]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Enable inline plotting
%matplotlib inline
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)  

Python version 2.7.13 (default, Dec 18 2016, 07:03:39) 
[GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)]
Pandas version 0.22.0
Matplotlib version 1.5.3


In [40]:
Location = r'train.csv'
df = pd.read_csv(Location, parse_dates=['booking_date', 'dep_date'])

df['org_dst_index'] = (df['origin'] + '_' + df['destination']).astype('category').cat.codes
maxIndex  = max(df['org_dst_index'])
df['org_dst_index'] = df['org_dst_index']/maxIndex
df['days'] = ((df['dep_date'] - df['booking_date'])/np.timedelta64(1, 'D')).astype(int)/365
df['dep_time_sc'] = ((df['dep_time']%100).astype(int)/5 + (df['dep_time']/100).astype(int)*12)/((24*12)-1)
df['weekday_dep'] = df['dep_date'].dt.dayofweek/6
df['month_book'] = df['booking_date'].dt.month/12
df['fare_choice_sc'] = df.groupby(['fare_choice']).ngroup()
df['org'] = df.groupby(['origin']).ngroup()
maxOrgIndex  = max(df['org'])
df['org'] = df['org']/maxOrgIndex
df['dst'] = df.groupby(['destination']).ngroup()
maxDstIndex  = max(df['dst'])
df['dst'] = df['dst']/maxDstIndex

flightDataPd = df[['org','dst','org_dst_index','pax', 'days','dep_time_sc','weekday_dep','month_book','fare_choice_sc']]
flightData = flightDataPd.as_matrix(columns=flightDataPd.columns[0:])
featureLength = 8
X = flightData[:,0:featureLength]
y = flightData[:,featureLength]
df.head()

,id,booking_date,origin,destination,dep_date,dep_time,pax,fare_choice,org_dst_index,days,dep_time_sc,weekday_dep,month_book,fare_choice_sc,org,dst
0,1,2017-02-10,MAA,DEL,2017-06-01,630,2,Deal,1.000000,0.304110,0.271777,0.5,0.166667,1,1.000000,0.75
1,2,2017-05-27,DEL,BOM,2017-06-01,1745,1,Classic,0.571429,0.013699,0.742160,0.5,0.416667,0,0.500000,0.25
2,3,2017-05-30,BOM,DEL,2017-06-01,900,1,Saver,0.285714,0.005479,0.376307,0.5,0.416667,3,0.166667,0.75
3,4,2017-05-30,BOM,BLR,2017-06-01,2130,2,Classic,0.142857,0.005479,0.898955,0.5,0.416667,0,0.166667,0.00
4,5,2017-05-30,DEL,BOM,2017-06-01,1240,1,Classic,0.571429,0.005479,0.529617,0.5,0.416667,0,0.500000,0.25


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifer = linear_model.LogisticRegression()
classifer.fit(X_train, y_train)
output_predicted = classifer.predict(X_test)
output_predicted_prob = classifer.predict_proba(X_test)
print(y_test)
print(output_predicted_prob)
print(output_predicted)
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, predicted))
print('Variance score: %.2f' % r2_score(y_test, predicted))


[3. 0. 3. ... 1. 1. 3.]
[[5.85010159e-01 1.10015770e-01 1.18724245e-01 1.86249826e-01]
 [6.47078213e-01 5.44868871e-02 1.40865969e-01 1.57568930e-01]
 [6.08738578e-01 9.08973153e-02 5.27779391e-02 2.47586168e-01]
 ...
 [1.13349460e-01 5.14673823e-01 9.24837807e-03 3.62728339e-01]
 [1.22001444e-03 7.98598038e-01 5.82125098e-04 1.99599823e-01]
 [3.43066306e-01 2.53033072e-01 5.39163171e-02 3.49984305e-01]]
[0. 0. 0. ... 1. 1. 3.]
Mean absolute error: 0.74
Variance score: -0.32
